**Imports**

In [1]:
import pyvisa
import tkinter as tk
import time
from tkinter import ttk

**PYVISA AND TCP DEVICE CONNECTION**

In [2]:
# Replace with the correct IP address and port of your probe bench
IP_ADDRESS = "192.168.3.1"
PORT = 10000  # Example port, replace with the correct one

rm = pyvisa.ResourceManager()
connection_string = f"TCPIP0::{IP_ADDRESS}::{PORT}::SOCKET"
# probe_bench = rm.open_resource(connection_string)
probe_bench = rm.open_resource('TCPIP0::192.168.3.1::10000::SOCKET')
probe_bench.read_termination = '\n'
probe_bench.write_termination = '\n'
probe_bench.timeout = 60000
print('Position Coords: ' + str(probe_bench.query('Cmd=1:31:Y C')))
# Check connection
# response = probe_bench.query("*IDN?")
print(f"Connected to: Prober")


Position Coords: Rsp=1:0:-107219.0 -106023.0 999.5 
Connected to: Prober


In [3]:
# probe_bench.close()

**DICTIONARY FOR TRANSLATING COMMAND STRING LITERALS**

In [4]:
# Dictionary for Chuck and Scope commands mapped to ASCII codes
# Dictionary mapping command names to their respective Command IDs
command_dict = {
    # Chuck Commands
    "InitChuck": '33',  
    # Initializes the chuck stage. Moves specified axis to negative end limit, then off end limit. Resets position counter to zero.
    # Parameters:
    # - Axis (optional byte): Bit 0x for X axis, Bit 1 for Y axis, Bit 2 for Z axis (default is All Axes)

    "MoveChuck": '34',  
    # Moves the chuck stage to the specified X, Y position relative to the wafer home position.
    # Parameters:
    # - xValueFloat: Target X position.
    # - yValueFloat: Target Y position.
    # - PosRefChar (optional): Reference position. 'H' for abs from home (default), 'Z' for abs from zero, 'C' for abs from center, 'R' for rel from current.
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.
    # - Velocity (optional): Movement velocity in percentage (default is 100%).

    "MoveChuckAlign": '38', 
    # Moves the chuck Z axis to the align height.
    # Parameters: None

    "MoveChuckContact": '37',
    # Moves the chuck Z axis to the contact height. In Search mode, it searches for contact height using edge sensor.
    # Parameters:
    # - Mode (optional): 'N' for normal mode, 'S' for search mode (default: N).
    # - Velocity (optional): Movement velocity in percentage (default is 100%).

    "MoveChuckLoad": '3A',  
    # Moves the chuck stage to the load position. Can optionally turn off the vacuum.
    # Parameters:
    # - TurnOffVacuum (optional byte): 1 to turn off vacuum, 0 to keep vacuum on (default is 0).

    "MoveChuckPosition": '3C',
    # General-purpose move command for the chuck stage, does not use Z mode.
    # Parameters:
    # - xValueFloat: Target X position.
    # - yValueFloat: Target Y position.
    # - PosRefChar (optional): Reference position. 'H' for abs from home (default), 'Z' for abs from zero, 'C' for abs from center, 'R' for rel from current.
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.
    # - Velocity (optional): Movement velocity in percentage (default is 100%).

    "MoveChuckVelocity": '3D',
    # - Moves the chuck stage in velocity mode. The motion continues until the StopChuckMovement command
    # is received, or the end limit (error condition) is reached.
    # - xValue: activate axis
    # - yValue: activate axis
    # - zValue: activate axis
    # - Velocity : %Velocity 

    "MoveChuckSeperation": '39', 
    # Moves the chuck Z axis to the separation height.
    # Parameters: None

    "MoveChuckZ": '3B',  
    # Moves the chuck Z axis to a specified height.
    # Parameters:
    # - zValueFloat: Target Z height.
    # - PosRefChar (optional): Reference position. 'Z' for abs from zero (default), 'H' for abs from contact, 'C' for abs from center, 'R' for rel from current.
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.
    # - Velocity (optional): Movement velocity in percentage (default is 100%).

    "ReadChuckHeights": '32',  
    # Returns the current settings for chuck Z movement (contact height, overtravel, align, separation, load).
    # Parameters:
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.

    "ReadChuckPosition": '31', 
    # Returns the current position of the chuck stage in X, Y, and Z.
    # Parameters:
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.
    # - PosRef (optional): Position reference. 'H' for from home (default), 'Z' for from zero, 'C' for from center.
    # - CompMode (optional): Compensation mode. 'M' for matrix comp, 'L' for linear comp only, 'N' for no comp.

    "ReadChuckStatus": '30',
    # Returns the current status of the chuck.
    # Parameters: None

    "SetChuckHeight": '42',
    # Defines predefined contact height and gaps for overtravel, align, load, and separation.
    # Parameters:
    # - Level: Contact height, overtravel, align, or separation height.
    # - Mode (optional): '0' for current position, 'R' for reset contact height, 'V' for take value.
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.
    # - Value (optional): Height value in specified units.

    "SetChuckHome": '40',
    # Sets the current chuck position as the wafer home position.
    # Parameters:
    # - Mode (optional): '0' for current position (default), 'V' for take value.
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.
    # - xValueFloat: X position.
    # - yValueFloat: Y position.

    "SetChuckMode": '3F',
    # Sets the chuck contact control mode.
    # Parameters:
    # - Overtravel (byte): 0=Off, 1=On, 2=Ignore.
    # - Auto Z (byte): 0=Off, 1=On, 2=Ignore.
    # - Interlock (byte): 0=Off, 1=On, 2=Ignore.
    # - Edge Sensor (byte): 0=Off, 1=On, 2=Ignore.
    # - Platen Stroke (byte): Reserved.

    "StopChuckMovement": '3E',
    # Stops all chuck movements in X, Y, and Z.
    # Parameters:
    # - Axis (optional byte): Bit 0x for X axis, Bit 1 for Y axis, Bit 2 for Z axis (default is all axes).

    # Scope Commands
    "InitScope": '73',  
    # Initializes the microscope stage in X, Y, and Z. Default is XY in minus and Z in plus.
    # Parameters:
    # - Axis (optional byte): Bit 0x for X axis, Bit 1 for Y axis, Bit 2 for Z axis (default is all axes).
    # - Direction (optional byte): Bit 00=X-, 1=X+, Bit 10=Y-, 1=Y+, Bit 20=Z-, 1=Z+.

    "MoveLiftDown": '77',
    # Moves the microscope lift to the lower position.
    # Parameters: None

    "MoveLiftUp": '76',
    # Moves the microscope lift to the upper position.
    # Parameters: None

    "MoveScope": '74',
    # Moves the microscope stage to the specified X, Y position relative to the home position.
    # Parameters:
    # - xValueFloat: Target X position.
    # - yValueFloat: Target Y position.
    # - PosRefChar (optional): Reference position. 'H' for abs from home (default), 'Z' for abs from zero, 'C' for abs from center, 'R' for rel from current.
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.
    # - Velocity (optional): Movement velocity in percentage (default is 100%).

    "MoveScopeIndex": '75',
    # Moves the microscope stage in index steps.
    # Parameters:
    # - xValueInt: X index steps.
    # - yValueInt: Y index steps.
    # - PositionRefChar (optional): Position reference. 'H' for abs from home (default), 'R' for rel from current.
    # - Velocity (optional): Movement velocity in percentage (default is 100%).

    "MoveScopePosition": '79',
    # General-purpose move command for the scope stage.
    # Parameters:
    # - xValueFloat: Target X position.
    # - yValueFloat: Target Y position.
    # - PosRefChar (optional): Reference position. 'H' for abs from home (default), 'Z' for abs from zero, 'C' for abs from center, 'R' for rel from current.
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.
    # - Velocity (optional): Movement velocity in percentage (default is 100%).

    "MoveScopeVelocity": '7A',
    # Moves the microscope stage in velocity mode.
    # Parameters:
    # - xAxisChar: Direction for X axis ('+' or '-' or '0').
    # - yAxisChar: Direction for Y axis ('+' or '-' or '0').
    # - zAxisChar: Direction for Z axis ('+' or '-' or '0').
    # - VelocityFloat: Movement velocity in percentage (max 100.0).

    "MoveScopeZ": '78',
    # Moves the microscope Z axis to a specified height.
    # Parameters:
    # - zValueFloat: Target Z height.
    # - PosRefChar (optional): Reference position. 'Z' for abs from zero (default), 'H' for abs from contact, 'C' for abs from center, 'R' for rel from current.
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.
    # - Velocity (optional): Movement velocity in percentage (default is 100%).

    "ReadScopePosition": '71',
    # Returns the current position of the microscope stage in X, Y, and Z.
    # Parameters:
    # - Unit (optional): Measurement unit. 'Y' microns (default), 'I' mils, 'E' encoder.
    # - PosRef (optional): Position reference. 'H' for from home (default), 'Z' for from zero, 'C' for from center.
    # - CompMode (optional): Compensation mode. 'M' for matrix comp, 'L' for linear comp only, 'N' for no comp.

    "ReadScopeStatus": '70',
    # Returns the current status of the microscope.
    # Parameters: A lot
}


# Example usage: sending a command to the probe bench
# def send_command(command_name):
#     if command_name in command_dict:
#         ascii_command = command_dict[command_name]
#         probe_bench.write_raw(ascii_command)  # Send the ASCII command directly
#     else:
#         print(f"Command {command_name} not found in the dictionary.")

# # Example: sending the "CHUCK_HOME" command
# send_command("CHUCK_HOME")


**Command Sending Methods**

In [5]:

def SendCommand(instrument, commandName, **kwargs):
    global Cmd
    # Access the parameters using the kwargs dictionary
    AxisByte = kwargs.get('AxisByte', None)
    xValue = kwargs.get('X', None)
    yValue = kwargs.get('Y', None)
    zValue = kwargs.get('Z', None)
    PosRefChar = kwargs.get('RefChar', None)
    Unit = kwargs.get('Unit', None)
    V = kwargs.get('V', None)
    Mode = kwargs.get('Mode', None)
    Velocity = kwargs.get('Velocity', None)
    CompMode = kwargs.get('CompMode', None)
    Level = kwargs.get('Level', None)
    Overtravel = kwargs.get('Overtravel', None)
    AutoZ = kwargs.get('AutoZ', None)
    Interlock = kwargs.get('Interlock', None)
    EdgeSensor = kwargs.get('EdgeSensor', None)
    Axis = kwargs.get('Axis', None)
    Direction = kwargs.get('Direction', None)
    
    if commandName in command_dict:
        # Retrieve the base command template
        asciiCommand = command_dict[commandName]
        
        # Initialize an empty list to hold command parts
        command_parts = []
        beginning = [f'Cmd={Cmd}:{asciiCommand}:']
        
        if commandName == 'InitChuck':
            command_parts.append(f'{AxisByte if AxisByte is not None else 7}')
        elif commandName == 'MoveChuck':
            command_parts.append(f'{xValue if xValue is not None else 0}')
            command_parts.append(f'{yValue if yValue is not None else 0}')
            command_parts.append(f'{PosRefChar if PosRefChar is not None else "H"}')
            command_parts.append(f'{Unit if Unit is not None else "Y"}')
            command_parts.append(f'{Velocity if Velocity is not None else 100}')
        elif commandName == 'MoveChuck Contact':
            command_parts.append(f'{Move if Mode is not None else "N"}')
            command_parts.append(f'{Velocity if Velocity is not None else 100}')
        elif commandName == 'MoveChuckPosition':
            command_parts.append(f'{xValue if xValue is not None else 0}')
            command_parts.append(f'{yValue if yValue is not None else 0}')
            command_parts.append(f'{PosRefChar if PosRefChar is not None else "H"}')
            command_parts.append(f'{Unit if Unit is not None else "Y"}')
            command_parts.append(f'{Velocity if Velocity is not None else 100}')
        elif commandName == 'MoveChuckVelocity':
            command_parts.append(f'{xValue if xValue is not None else 0}')
            command_parts.append(f'{yValue if yValue is not None else 0}')
            command_parts.append(f'{zValue if zValue is not None else 0}')
            command_parts.append(f'{Velocity if Velocity is not None else 100}')
        elif commandName == 'MoveChuckZ':
            command_parts.append(f'{zValue if zValue is not None else 7000}')
            command_parts.append(f'{PosRefChar if PosRefChar is not None else "Z"}')
            command_parts.append(f'{Unit if Unit is not None else "Y"}')
            command_parts.append(f'{Velocity if Velocity is not None else 100}')
        elif commandName == 'ReadChuckHeights':
            command_parts.append(f'{Unit if Unit is not None else "Y"}')
        elif commandName == 'ReadChuckPosition':
            command_parts.append(f'{Unit if Unit is not None else "Y"}')
            command_parts.append(f'{PosRefChar if PosRefChar is not None else "H"}')
            command_parts.append(f'{CompMode if CompMode is not None else "N"}')
        elif commandName == 'SetChuckHeight':
            command_parts.append(f'{Level if Level is not None else "C"}')
            command_parts.append(f'{Mode if Mode is not None else 0}')
            command_parts.append(f'{Unit if Unit is not None else "Y"}')
            if Value is not None:
                command_parts.append(f'{Value}')
        elif commandName == 'SetChuckHome':
            command_parts.append(f'{Mode if Mode is not None else 0}')
            command_parts.append(f'{Unit if Unit is not None else "Y"}')
            if Mode == 'V':
                command_parts.append(f'{xValue}')
                command_parts.append(f'{yValue}')
        elif commandName == 'SetChuckMode':
            command_parts.append(f'{Overtravel if Overtravel is not None else 0}')
            command_parts.append(f'{AutoZ if AutoZ is not None else 0}')
            command_parts.append(f'{Interlock if Interlock is not None else 0}')
            command_parts.append(f'{EdgeSensor if EdgeSensor is not None else 0}')
        elif commandName == 'StopChuckMovement':
            command_parts.append(f'{AxisByte if AxisByte is not None else 7}')
    
            
        almost_command = ' '.join(command_parts)
        beginning.append(almost_command)
        full_command = ''.join(beginning)
        # Send the complete command
        print(full_command)
        if commandName in ('ReadChuckPosition', 'ReadChuckHeights', 'ReadChuckStatus'):
            print('Send Command with important Read')
            Response = str(instrument.query(full_command))
            print(Response)
            Response = Response.strip('\r')
            # Split the response into parts based on the colon separator
            parts = Response.split(':')
            if parts[2] == '':
                time.sleep(1)
                Response = str(instrument.query(full_command))
                print(Response)
                Response = Response.strip('\r')
                # Split the response into parts based on the colon separator
                parts = Response.split(':')
            
            # Extract the error byte, which is the second part
            error_code = parts[1]
            DataBytes = parts[2].strip().split(' ')
            # Check if the error code is 0
            if error_code == '0':
                # No error, return Error = False and each byte separately
                error = 0
                if commandName == 'ReadChuckStatus':
                    DataBytes = DataBytes[:-2]
                byte_values = DataBytes[0:]  # Skip the first part which is "Rsp=1"
                print(byte_values)
                print(error)
            else:
                # Error present, return Error = True and '0' for each byte
                error = error_code
                byte_values = ['0'] * (len(parts) - 1)

            # Prepare the return tuple, including the error and each byte
            Cmd += 1
            return (error, *byte_values)
        else:
            print('Send Command without important Read')
            Response = str(instrument.query(full_command))
            print(Response)
#             Split the response into parts based on the colon separator
            parts = Response.split(':')
            
            # Extract the error byte, which is the second part
            error_code = parts[1] + parts[2]
            Cmd += 1
            if error_code == '0':
                return 0
            else:
                return error_code
        
    else:
        print(f"Command {commandName} not found in command_dict")
    
    return

def ChuckSetup():
    SendCommand(probe_bench, 'InitChuck', AxisByte = 7)
    time.sleep(20)
    SendCommand(probe_bench, 'MoveChuckLoad')
    time.sleep(10)
    return

def ScopeSetup(): #needs work fix the ReadScopeStatus
#     SendCommand(probe_bench, 'InitScope')
#     SendCommand(probe_bench, 'SetScopeMode')
    return 







# #Main()
Cmd = 1
ChuckSetup()
# SendCommand(probe_bench, 'StopChuckMovement')
# SendCommand(probe_bench, 'MoveChuck', X = 1000, Y = 1500, Unit = 'I')

Cmd=1:33:7
Send Command without important Read
Rsp=1:0:
Cmd=2:3A:
Send Command without important Read
Rsp=2:0:


**Methods and Class Definition For GUI**

In [6]:
def move_position():
    X = x_entry.get()
    Y = y_entry.get()
    Z = z_entry.get()

    Error, IsInit, zMode, onEndLimit, isMoving, CompMode, Vacuum, ZPos = SendCommand(probe_bench, 'ReadChuckStatus')

    while isMoving != '0':
        time.sleep(0.1)  # Add a 100 ms delay
        Error, IsInit, zMode, onEndLimit, isMoving, CompMode, Vacuum, ZPos = SendCommand(probe_bench, 'ReadChuckStatus')
    print(X, Y, Z)
    Error = SendCommand(probe_bench, 'MoveChuck', X = X, Y = Y)
    print(f'MoveChuck Error: {Error}')
    Error = SendCommand(probe_bench, 'MoveChuckZ', Z = Z)
    print(f'MoveChuckZ Error: {Error}')
    
def get_position():
    Error, X, Y, Z = SendCommand(probe_bench, 'ReadChuckPosition')
    Z = float(Z) + 20014.75
    X = float(X) + 6
    if Error == 0:
        x_pos.delete(0, tk.END)
        x_pos.insert(0, X)
        y_pos.delete(0, tk.END)
        y_pos.insert(0, Y)
        z_pos.delete(0, tk.END)
        z_pos.insert(0, Z)

def setup_device():
    # Disable only the selected check buttons
    Interlock = 0
    AutoZ = 0
    EdgeSensor = 0
    if interlock_var.get() == 1:
        interlock_radio.config(state=tk.DISABLED)
        #add to list of settings to turn on
        InterLock = 1
    if autoz_var.get() == 1:
        autoz_radio.config(state=tk.DISABLED)
        #add to list of settings to turn on
        AutoZ = 1
    if edge_sensor_var.get() == 1:
        edge_sensor_radio.config(state=tk.DISABLED)
        #add to list of settings to turn on
        EdgeSensor = 1
    #use list to send command to prober
    SendCommand(probe_bench, 'SetChuckMode', Overtravel = 0, AutoZ = AutoZ, Interlock = InterLock, EdgeSensor = EdgeSensor)

# Create the main window
root = tk.Tk()
root.title("Probe Station GUI")

# Move Position Line
move_frame = ttk.Frame(root, padding="10")
move_frame.grid(row=0, column=0, sticky=tk.W)

ttk.Label(move_frame, text="X:").grid(row=0, column=0)
x_entry = ttk.Entry(move_frame, width=10)
x_entry.grid(row=0, column=1)

ttk.Label(move_frame, text="Y:").grid(row=0, column=2)
y_entry = ttk.Entry(move_frame, width=10)
y_entry.grid(row=0, column=3)

ttk.Label(move_frame, text="Z:").grid(row=0, column=4)
z_entry = ttk.Entry(move_frame, width=10)
z_entry.grid(row=0, column=5)

go_button = ttk.Button(move_frame, text="Go", command=move_position)
go_button.grid(row=0, column=6)

# Get Position Line
get_frame = ttk.Frame(root, padding="10")
get_frame.grid(row=1, column=0, sticky=tk.W)

get_button = ttk.Button(get_frame, text="Get", command=get_position)
get_button.grid(row=0, column=0)

x_pos = ttk.Entry(get_frame, width=10)
x_pos.grid(row=0, column=1)
y_pos = ttk.Entry(get_frame, width=10)
y_pos.grid(row=0, column=2)
z_pos = ttk.Entry(get_frame, width=10)
z_pos.grid(row=0, column=3)

# Setup Line
setup_frame = ttk.Frame(root, padding="10")
setup_frame.grid(row=2, column=0, sticky=tk.W)

interlock_var = tk.IntVar()
autoz_var = tk.IntVar()
edge_sensor_var = tk.IntVar()

interlock_radio = ttk.Checkbutton(setup_frame, text="Interlock", variable=interlock_var)
interlock_radio.grid(row=0, column=0, sticky=tk.W)

autoz_radio = ttk.Checkbutton(setup_frame, text="AutoZ", variable=autoz_var)
autoz_radio.grid(row=0, column=1, sticky=tk.W)

edge_sensor_radio = ttk.Checkbutton(setup_frame, text="Edge Sensor", variable=edge_sensor_var)
edge_sensor_radio.grid(row=0, column=2, sticky=tk.W)

setup_button = ttk.Button(setup_frame, text="Setup", command=setup_device)
setup_button.grid(row=1, column=0, columnspan=3)

**RUN THIS FOR USE (MAIN GUI)**

In [ ]:
# Run the application
root.mainloop()

Cmd=3:31:Y H N
Send Command with important Read
Rsp=3:0:-6.0 0.0 -14514.75 
['-6.0', '0.0', '-14514.75']
0
Cmd=4:30:
Send Command with important Read
Rsp=4:0:7 22 0 0 T 0 0 0 0 
['7', '22', '0', '0', 'T', '0', '0']
0
6000 200000 9000
Cmd=5:34:6000 200000 H Y 100
Send Command without important Read
Rsp=5:0:
MoveChuck Error: 0
Cmd=6:3B:9000 Z Y 100
Send Command without important Read
Rsp=6:0:
MoveChuckZ Error: 0
Cmd=7:31:Y H N
Send Command with important Read
Rsp=7:0:6003.0 199999.5 -11014.75 
['6003.0', '199999.5', '-11014.75']
0
Cmd=8:31:Y H N
Send Command with important Read
Rsp=8:0:6002.5 199999.0 -11014.75 
['6002.5', '199999.0', '-11014.75']
0
Cmd=9:31:Y H N
Send Command with important Read
Rsp=9:0:6002.5 199999.0 -11014.75 
['6002.5', '199999.0', '-11014.75']
0
Cmd=10:31:Y H N
Send Command with important Read
Rsp=10:0:6002.5 199999.0 -11014.75 
['6002.5', '199999.0', '-11014.75']
0
Cmd=11:31:Y H N
Send Command with important Read
Rsp=11:0:6002.5 199999.0 -11014.75 
['6002.5', '199